In [60]:
import pickle as pkl
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import collections

In [51]:
file = open('./train_comments_subs.pkl', 'rb')
data = pkl.load(file)
data, len(data)

([{'id': '00004320bb',
   'ingredients': [['boiling_water'],
    ['cool_whip', 'french_vanilla_cool_whip'],
    ['seedless_watermelon'],
    ['graham_cracker_crust']],
   'subs': ('seedless_watermelon', 'lime')},
  {'id': '0000631d90',
   'ingredients': [['shredded_coconut'],
    ['lean_ground_beef'],
    ['fresh_garlic_clove', 'fresh_garlic_cloves', 'fresh_garlic'],
    ['salt_and_black_pepper'],
    ['lemon_juice'],
    ['soy_sauce'],
    ['cornstarch'],
    ['pineapple_chunk', 'pineapple_chunks', 'pineapple_topping'],
    ['mandarin_orange',
     'mandarin_oranges',
     'mandarin_oranges_in_light_syrup',
     'satsuma_oranges'],
    ['cashew_nut', 'cashew_nuts']],
   'subs': ('lean_ground_beef', 'chicken_breast')},
  {'id': '0000ed95f8',
   'ingredients': [['butter', 'molly_mcbutter'],
    ['brown_sugar'],
    ['egg', 'eggs', 'duck_egg'],
    ['orange_juice_concentrate'],
    ['orange_rind'],
    ['flour'],
    ['baking_soda', 'baking_ammonia'],
    ['salt', 'vegetable_salt', 'low_

In [11]:
df = pd.read_csv('./dataset/full_dataset.csv')
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [17]:
vocab = []
with open('./test.npy', 'rb') as f:
    vocab = np.load(f, allow_pickle=True)

In [19]:
ingredients = [row[1] for row in vocab[1:-1]]
ingredients

[['1%_fat_buttermilk', 'low_fat_buttermilk'],
 ['1%_fat_cottage_cheese'],
 ['10%_cream', '15%_cream'],
 ['100%_bran'],
 ['10_inch_flour_tortilla',
  '10_inch_flour_tortillas',
  '12_inch_flour_tortilla'],
 ['12_inch_pizza_crust'],
 ['18%_table_cream'],
 ['2%_buttermilk'],
 ['2%_cheddar_cheese'],
 ['2%_evaporated_milk'],
 ['2%_fat_cottage_cheese'],
 ['2%_milk', '2%_low_fat_milk'],
 ['2%_mozzarella_cheese'],
 ['35%_cream'],
 ['6_inch_corn_tortilla', '6_inch_corn_tortillas'],
 ['6_inch_flour_tortilla', '6_inch_flour_tortillas'],
 ['7_up', 'cherry_7_up', '7_up_soda', 'diet_7_up'],
 ['80%_lean_ground_beef'],
 ['85%_lean_ground_beef'],
 ['8_inch_flour_tortilla', '8_inch_flour_tortillas', '7_inch_flour_tortilla'],
 ['9"_pastry_pie_shell', '9"_pastry_pie_shells'],
 ['9"_unbaked_pie_shell', '9"_unbaked_pie_shells'],
 ['90%_lean_ground_beef'],
 ['93%_lean_ground_beef'],
 ['95%_lean_ground_beef'],
 ['96%_lean_ground_beef'],
 ['98%_fat_free_cream_of_chicken_soup'],
 ['98%_fat_free_cream_of_mushroo

In [29]:
nerSet = set() 
for recipe in df['NER']:
  for ingredient in ast.literal_eval(recipe):
    nerSet.add(ingredient)
nerSet

{'Shortbread cookies',
 'Peanut Putter',
 'atomb cheese',
 'low-fat organic buttermilk',
 'Chocolate covered almonds',
 'Frenchie Connection',
 'your favorite homemade',
 'all-purpose unbleached flour',
 'Lemon -or possibly-',
 'avocados bite',
 'TAST',
 'containers vanilla pudding',
 'Handful flat leaf parsley',
 'herb stock',
 'fresh kale greens',
 'chipped ham',
 'Amarula cream',
 'Chocolate Chip/Walnuts',
 'Tomato Chopped',
 'Picholine olives',
 'stone ground mustard',
 'BRANDIED BUTTERCREAM',
 'or larger Oreo cookies',
 'containers applesauce',
 'favorite cheese sauce',
 'red ground cayenne pepper',
 'brushing',
 'Fluffy white rice',
 'white butterfish',
 'Junkets',
 'approximate',
 'strawberry-flavor',
 'regular M',
 'drizzle of extra-virgin olive',
 'thin parsnips',
 'n Chedder sausages',
 'salsa SWEET',
 'multicolored cherry tomatoes',
 'Galiano',
 'while',
 'BROWNIE BATTER',
 'water(Check notes',
 'Absolut Mandarin',
 'pounds ripe',
 'pealings',
 'chocolate pistoles',
 'Pork m

In [30]:
len(nerSet)

234059

In [38]:
len(ingredients)

6632

In [37]:
mappedVocab = {}
for entity in ingredients:
  for ingredient in entity:
    if ingredient in nerSet:
      mappedVocab[str(entity)] = ingredient
      break
mappedVocab, len(mappedVocab)

({"['abalone']": 'abalone',
  "['absinthe']": 'absinthe',
  "['achiote']": 'achiote',
  "['acorn', 'acorns']": 'acorn',
  "['advocaat']": 'advocaat',
  "['agar']": 'agar',
  "['agave']": 'agave',
  "['ahi']": 'ahi',
  "['aioli']": 'aioli',
  "['ajinomoto']": 'ajinomoto',
  "['ajwain']": 'ajwain',
  "['alcaparrado']": 'alcaparrado',
  "['alcohol']": 'alcohol',
  "['ale']": 'ale',
  "['alfalfa']": 'alfalfa',
  "['allspice']": 'allspice',
  "['almond', 'almonds']": 'almond',
  "['alum']": 'alum',
  "['amaranth']": 'amaranth',
  "['amaretti']": 'amaretti',
  "['amaretto']": 'amaretto',
  "['ammonia']": 'ammonia',
  "['anchovy', 'anchovies']": 'anchovy',
  "['angelica']": 'angelica',
  "['anise']": 'anise',
  "['anisette']": 'anisette',
  "['apple', 'apples', 'winesap_apples', 'apple_ring']": 'apple',
  "['applejack']": 'applejack',
  "['applesauce']": 'applesauce',
  "['apricot', 'apricots', 'canned_apricots']": 'apricot',
  "['aquavit']": 'aquavit',
  "['armagnac']": 'armagnac',
  "['arro

In [58]:
matchedRecipes = []
for recipe in data:
  temp = True
  for entity in recipe['ingredients']:
    if not str(entity) in mappedVocab:
      temp = False
  if temp:
    matchedRecipes.append([mappedVocab[str(entity)] for entity in recipe['ingredients']])
    # for ingredient in entity
    # print(ingredient)
matchedRecipes, len(matchedRecipes)

([['rice', 'milk', 'egg', 'sugar', 'vanilla', 'raisin', 'nutmeg', 'cinnamon'],
  ['carrot', 'water', 'butter', 'sugar', 'salt'],
  ['flour', 'milk', 'shortening', 'margarine', 'sugar', 'vanilla', 'salt'],
  ['egg', 'water'],
  ['rice', 'salt', 'water', 'butter'],
  ['butter', 'milk', 'egg', 'pepper', 'salt', 'bacon', 'cheese'],
  ['water', 'margarine', 'salt', 'egg'],
  ['flour', 'salt', 'sugar', 'water', 'yeast', 'egg', 'oil'],
  ['oil', 'onion', 'flour', 'egg', 'water', 'vanilla', 'plantain'],
  ['egg', 'salt', 'water', 'butter', 'flour'],
  ['bagel', 'egg', 'milk', 'vanilla', 'cinnamon', 'salt'],
  ['cocoa', 'flour', 'sugar', 'milk', 'butter', 'vanilla'],
  ['milk', 'cocoa', 'honey', 'salt', 'paprika'],
  ['cabbage', 'butter', 'flour', 'salt', 'milk', 'breadcrumb'],
  ['butter', 'salt', 'egg', 'milk', 'flour', 'shortening'],
  ['potato', 'onion', 'water'],
  ['strawberry', 'sugar', 'water'],
  ['honey', 'lemon', 'water'],
  ['honey', 'lemon', 'water'],
  ['juice', 'flour', 'cereal']

In [73]:
sameRecipes = []
for recipe in matchedRecipes:
  recipeSet = set(recipe)
  for nerRecipe in df['NER'][:10000]:
    mappedNerRecipe = ast.literal_eval(nerRecipe)
    if len(mappedNerRecipe) == len(recipe):
      if len(list(set(mappedNerRecipe) - recipeSet)) == 1:
        sameRecipes.append((recipe, mappedNerRecipe))
sameRecipes, len(sameRecipes)

([(['carrot', 'water', 'butter', 'sugar', 'salt'],
   ['water', 'yeast', 'butter', 'sugar', 'salt']),
  (['carrot', 'water', 'butter', 'sugar', 'salt'],
   ['flour', 'butter', 'sugar', 'salt', 'water']),
  (['flour', 'milk', 'shortening', 'margarine', 'sugar', 'vanilla', 'salt'],
   ['milk', 'butter', 'sugar', 'salt', 'flour', 'shortening', 'vanilla']),
  (['egg', 'water'], ['whole wheat flour', 'water']),
  (['egg', 'water'], ['masa harina', 'water']),
  (['egg', 'water'], ['water', 'sugar']),
  (['egg', 'water'], ['glycerine', 'water']),
  (['egg', 'water'], ['water', 'chicken']),
  (['egg', 'water'], ['cake mix', 'egg']),
  (['egg', 'water'], ['pineapple juice', 'water']),
  (['egg', 'water'], ['broccoli', 'water']),
  (['rice', 'salt', 'water', 'butter'],
   ['water', 'long grain rice', 'salt', 'butter']),
  (['flour', 'salt', 'sugar', 'water', 'yeast', 'egg', 'oil'],
   ['water', 'oil', 'sugar', 'yeast', 'salt', 'eggs', 'flour']),
  (['flour', 'salt', 'sugar', 'water', 'yeast', 'e